In [32]:
!pip install -q yfinance pandas numpy matplotlib scikit-learn keras gradio flask fpdf2 beautifulsoup4 requests transformers pyTelegramBotAPI gTTS
# Required libraries install kar raha hai (finance, ML, visualization, NLP, chatbot, PDF, voice).

"""
Yeh sab required libraries install karta hai:

yfinance → stock data fetch

pandas/numpy → data handling

matplotlib → graph plotting

scikit-learn → scaling

keras → LSTM neural network

gradio → web UI

flask → API endpoint

fpdf2 → PDF report

bs4 + requests → news scraping

transformers → FinBERT model for sentiment

pyTelegramBotAPI → Telegram alerts

gTTS → text-to-speech
"""


'\nYeh sab required libraries install karta hai:\n\nyfinance → stock data fetch\n\npandas/numpy → data handling\n\nmatplotlib → graph plotting\n\nscikit-learn → scaling\n\nkeras → LSTM neural network\n\ngradio → web UI\n\nflask → API endpoint\n\nfpdf2 → PDF report\n\nbs4 + requests → news scraping\n\ntransformers → FinBERT model for sentiment\n\npyTelegramBotAPI → Telegram alerts\n\ngTTS → text-to-speech\n'

In [33]:
import yfinance as yf       # stock data download
import pandas as pd         # data handling
import numpy as np          # numerical calculations
import matplotlib.pyplot as plt  # plotting graphs
import gradio as gr         # web UI
import requests             # fetch webpages
from bs4 import BeautifulSoup  # web scraping
from sklearn.preprocessing import MinMaxScaler  # data normalization
from keras.models import Sequential             # NN container
from keras.layers import LSTM, Dense            # LSTM layers for stock prediction
from fpdf import FPDF                          # PDF report generator
import datetime
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification  # FinBERT for news sentiment
import os, sqlite3
import telebot
from flask import Flask, request, jsonify, send_file
from gtts import gTTS       # Text-to-speech (voice summary)


In [35]:
scaler = MinMaxScaler(feature_range=(0, 1))
MODEL_NAME = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
label_map = {0: "negative", 1: "neutral", 2: "positive"}
DB_FILE = "stock_predictions.db"
BOT_TOKEN = "YOUR_TELEGRAM_BOT_TOKEN"
CHAT_ID = "YOUR_CHAT_ID"

"""
Scaler → data ko 0–1 ke range me laya jata hai.

FinBERT model → financial sentiment analysis ke liye pretrained model.

DB_FILE → SQLite database file.

BOT_TOKEN & CHAT_ID → Telegram alert ke liye.
"""



'\nScaler → data ko 0–1 ke range me laya jata hai.\n\nFinBERT model → financial sentiment analysis ke liye pretrained model.\n\nDB_FILE → SQLite database file.\n\nBOT_TOKEN & CHAT_ID → Telegram alert ke liye.\n'

In [36]:
def init_db():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS predictions
                 (ticker TEXT, predicted_price REAL, date TEXT)''')
    conn.commit()
    conn.close()
"""
Agar database nahi hai to predictions table create hota hai.
"""

'\nAgar database nahi hai to predictions table create hota hai.\n'

In [37]:
def save_to_db(ticker, price):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("INSERT INTO predictions VALUES (?, ?, ?)", (ticker, price, datetime.datetime.now().strftime('%Y-%m-%d')))
    conn.commit()
    conn.close()

  # Har prediction DB me save hoti hai.



In [38]:
def get_stock_data(ticker):
    df = yf.download(ticker, period="1y")
    return df[['Close']]

    # Yahoo Finance se 1 saal ka closing price data fetch hota hai.


In [39]:
def prepare_data(data):
    data_scaled = scaler.fit_transform(data)
    x, y = [], []
    for i in range(60, len(data_scaled)):
        x.append(data_scaled[i-60:i, 0])  # past 60 days
        y.append(data_scaled[i, 0])       # next day
    return np.array(x), np.array(y)

# Sliding window method – last 60 days input, agla din output.

In [40]:
def build_model():
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(60, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Simple 2-layer LSTM + Dense output model.

In [41]:
def predict_stock(ticker):
    df = get_stock_data(ticker)
    if df.empty:
        raise Exception("Invalid stock ticker or no data available.")
    x, y = prepare_data(df.values)
    x = x.reshape((x.shape[0], x.shape[1], 1))
    model = build_model()
    model.fit(x, y, epochs=5, batch_size=32, verbose=0)
    latest_60 = scaler.transform(df[-60:].values)
    input_data = latest_60.reshape(1, 60, 1)
    predicted_price = model.predict(input_data)[0][0]
    predicted_price = scaler.inverse_transform([[predicted_price]])[0][0]
    return round(predicted_price, 2), df

# Training + prediction → predicted closing price nikalta hai.

In [42]:
def fetch_news(ticker):
    search_url = f"https://www.google.com/search?q={ticker}+stock+news"
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        page = requests.get(search_url, headers=headers, timeout=10)
        soup = BeautifulSoup(page.content, "html.parser")
        news_items = soup.select('div.BNeawe.vvjwJb.AP7Wnd')
        return [item.get_text() for item in news_items[:5]]
    except:
        return ["Unable to fetch news at the moment."]

# Google news scrape, top 5 headlines.

In [43]:
def analyze_news(news_list):
    results = []
    for news in news_list:
        inputs = tokenizer(news, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = sentiment_model(**inputs)
            scores = torch.nn.functional.softmax(outputs.logits, dim=1)
            label_idx = torch.argmax(scores).item()
            confidence = scores[0][label_idx].item()
        results.append((news, label_map[label_idx], confidence))
    return results
# FinBERT model se sentiment analysis (positive, neutral, negative).

In [44]:
def create_chart(df, ticker):
    plt.figure(figsize=(10, 4))
    df['Close'].plot(title=f"{ticker.upper()} Closing Price (Past Year)")
    chart_file = f"{ticker}_chart.png"
    plt.savefig(chart_file)
    plt.close()
    return chart_file

# Stock ka chart banta hai aur image save hoti hai.

In [45]:
def create_pdf(ticker, predicted_price, news_analysis, chart_path):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=f"Stock Report for {ticker.upper()} - {datetime.datetime.now().strftime('%Y-%m-%d')}", ln=True)
    pdf.cell(200, 10, txt=f"Predicted Price: ${predicted_price}", ln=True)
    pdf.image(chart_path, w=180)
    pdf.cell(200, 10, txt="News Sentiment Analysis:", ln=True)
    for news, label, score in news_analysis:
        pdf.multi_cell(0, 10, f"- {news}\nSentiment: {label.upper()} (Confidence: {round(score, 2)})\n")
    filename = f"{ticker}_report.pdf"
    pdf.output(filename)
    return filename

# PDF banata hai: prediction + chart + news sentiment.

In [46]:
def speak_summary(ticker, predicted_price):
    message = f"The predicted price for {ticker.upper()} is {predicted_price} dollars."
    tts = gTTS(text=message, lang='en')
    filename = f"{ticker}_summary.mp3"
    tts.save(filename)
    return filename

# MP3 voice file generate karta hai.

In [47]:
def send_telegram_alert(ticker, price):
    try:
        bot = telebot.TeleBot(BOT_TOKEN)
        msg = f"Prediction Alert!\n{ticker.upper()} expected price: ${price}"
        bot.send_message(CHAT_ID, msg)
    except:
        print("Telegram alert failed. Check your token or chat ID.")

# Telegram bot pe alert bhejta hai.

In [48]:
def full_analysis(ticker):
    try:
        predicted_price, df = predict_stock(ticker)
        news = fetch_news(ticker)
        analysis = analyze_news(news)
        chart_path = create_chart(df, ticker)
        pdf_path = create_pdf(ticker, predicted_price, analysis, chart_path)
        voice_path = speak_summary(ticker, predicted_price)
        save_to_db(ticker, predicted_price)
        send_telegram_alert(ticker, predicted_price)
        return f"{ticker.upper()} predicted price: ${predicted_price}", pdf_path, voice_path
    except Exception as e:
        return f"Error: {str(e)}", None, None

#Sab modules ek jagah execute karta hai → prediction + PDF + voice + DB + Telegram.

In [49]:

iface = gr.Interface(
    fn=full_analysis,
    inputs=gr.Textbox(label="Enter Stock Ticker (e.g., AAPL)", placeholder="Type a stock symbol..."),
    outputs=[
        gr.Text(label="Prediction Result"),
        gr.File(label="Download PDF Report"),
        gr.File(label="Voice Summary (MP3)")
    ],
    title="Stock Market AI + Voice + Alerts",
    description="LSTM stock predictor + FinBERT news sentiment + PDF + Chart + Voice summary + Telegram alert + SQLite logging"
)

# Simple web app jaha ticker dal kar analysis run hota hai.

In [50]:
app = Flask(__name__)

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.json
    ticker = data.get('ticker')
    if not ticker:
        return jsonify({"error": "No ticker provided"})
    prediction, pdf_path, voice_path = full_analysis(ticker)
    return jsonify({"prediction": prediction, "pdf_path": pdf_path, "voice_path": voice_path})

@app.route('/download/<filename>')
def download_file(filename):
    if os.path.exists(filename):
        return send_file(filename, as_attachment=True)
    else:
        return "File not found", 404

# Flask API banata hai jo external requests handle karega.

In [51]:
init_db()
iface.launch(share=True)

# DB init + Gradio launch.

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1264e21e193292dfe7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7a9c539cf3ea5ad8b6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
